In [1]:
import numpy as np
import open3d as o3d  # type: ignore
import pymeshfix # type: ignore
from scipy.spatial import cKDTree # type: ignore

from superprimitive_fusion.mesh_fusion_utils import (
    smooth_normals,
    calc_local_spacing,
    compute_overlap_set_cached,
    smooth_overlap_set_cached,
    precompute_cyl_neighbours,
    update_weights,
    normal_shift_smooth,
    find_boundary_edges,
    topological_trim,
    merge_nearby_clusters,
    get_mesh_components,
)
from superprimitive_fusion.mesh_fusion import (
    fuse_meshes,
    sanitise_mesh,
)
from superprimitive_fusion.scanner import (
    capture_spherical_scans,
)
from superprimitive_fusion.utils import (
    bake_uv_to_vertex_colours,
    polar2cartesian,
)

In [2]:
mesh_gt = o3d.io.read_triangle_mesh("../data/mustard-bottle/textured.obj", enable_post_processing=True)

bake_uv_to_vertex_colours(mesh_gt)

mesh_gt.compute_vertex_normals()

bb = mesh_gt.get_minimal_oriented_bounding_box()
scale = np.mean(bb.get_max_bound())

In [ ]:
o3d.visualization.draw_geometries([mesh_gt])

In [3]:
scans = capture_spherical_scans(
    meshlist=[mesh_gt],
    num_views=6,
    radius=0.3,
    width_px=360,
    height_px=240,
    fov=70,
    k=5,
    sampler="fibonacci",
)

meshes = [scan['mesh'][0][0] for scan in scans]
weights = [scan['mesh'][1][0] for scan in scans]

o3d.visualization.draw_geometries(
    meshes,
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

/Users/christopher/Documents/Coding/Imperial/MSc Project/MSc-Project/src/superprimitive_fusion/scanner.py:548: RuntimeWarning: invalid value encountered in matmul
  z = ((points - cam_centre) @ L).clip(min=0.0) # (H,W)


TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: list[open3d.cpu.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False, lookat: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, up: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, front: Optional[numpy.ndarray[numpy.float64[3, 1]]] = None, zoom: Optional[float] = None) -> None

Invoked with: [[TriangleMesh with 10416 points and 20071 triangles.], [TriangleMesh with 7198 points and 13855 triangles.], [TriangleMesh with 5597 points and 10847 triangles.], [TriangleMesh with 5988 points and 11411 triangles.], [TriangleMesh with 10371 points and 20130 triangles.], [TriangleMesh with 9113 points and 17662 triangles.]]; kwargs: window_name='Virtual scan', front=[0.3, 1, 0], lookat=[0, 

In [4]:
from superprimitive_fusion.scanner import virtual_mesh_scan

In [5]:
meshes, _ = virtual_mesh_scan(
    meshlist=[mesh_gt],
    cam_centre=mesh_gt.get_center() + polar2cartesian(0.4, 60, 45),#45,150?
    look_at=mesh_gt.get_center(),
    k=10,
    max_normal_angle_deg=None,
    linear_depth_sigma=0,
    quadrt_depth_sigma=0,
    sigma_floor=0,
    bias_k1=0.03,
)
mesh1 = meshes[0][0]
weights1 = meshes[1][0]
# o3d.visualization.draw_geometries([mesh1])

meshes, _ = virtual_mesh_scan(
    meshlist=[mesh_gt],
    cam_centre=mesh_gt.get_center() + polar2cartesian(0.4, 60, 150),#45,150?
    look_at=mesh_gt.get_center(),
    k=10,
    max_normal_angle_deg=None,
    linear_depth_sigma=0,
    quadrt_depth_sigma=0,
    sigma_floor=0,
    bias_k1=0.03
)
mesh2 = meshes[0][0]
weights2 = meshes[1][0]
# o3d.visualization.draw_geometries([mesh2])

o3d.visualization.draw_geometries([mesh1, mesh2])

/Users/christopher/Documents/Coding/Imperial/MSc Project/MSc-Project/src/superprimitive_fusion/scanner.py:721: RuntimeWarning: divide by zero encountered in divide
  w[valid] = (1.0 / (sigma_z ** 2)).astype(np.float32)


In [6]:
weights1 = np.ones_like(weights1, dtype=np.float64)
weights2 = np.ones_like(weights2, dtype=np.float64)

In [ ]:
# j = 6
# fused_mesh = scans[0]
# for i in range(1,j):
#     fused_mesh = fuse_meshes(
#         fused_mesh,
#         scans[i],
#         h_alpha=3,
#         nrm_shift_iters=2,
#         shift_all=False,
#         fill_holes=True,
#     )

# o3d.visualization.draw_geometries(
#     [fused_mesh],
#     window_name="Virtual scan",
#     front=[0.3, 1, 0],
#     lookat=[0, 0, 0],
#     up=[0, 0, 1],
#     zoom=0.7,
# )

In [ ]:
# show_connected_components(fused_mesh)

In [7]:
# mesh1 = o3d.io.read_triangle_mesh("meshes/bottle_1.ply")
# mesh2 = o3d.io.read_triangle_mesh("meshes/bottle_2.ply")
# mesh1 = meshes[0]
# mesh2 = meshes[1]
# Mesh1 must be the already fused mesh
# weights1 = weights[0]
# weights2 = weights[1]
sigma_theta = 0.5
h_alpha: float = 3.0
r_alpha: float = 2.0
nrm_shift_iters: int = 2
nrm_smth_iters: int = 1
shift_all: bool = False

In [39]:
import copy
m01 = copy.deepcopy(mesh1)
m01.compute_vertex_normals()
o3d.visualization.draw_geometries([m01])

In [9]:
import copy
m1 = copy.deepcopy(mesh1)
m1.paint_uniform_color([1,0,0])
m2 = copy.deepcopy(mesh2)
m2.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([m1, m2])

In [8]:
# ---------------------------------------------------------------------
# Raw geometry & attribute extraction
# ---------------------------------------------------------------------
points1 = np.asarray(mesh1.vertices)
points2 = np.asarray(mesh2.vertices)

pointclouds = (points1, points2)
points = np.vstack(pointclouds)

assert weights1.ndim == 1 and len(weights1) == len(mesh1.vertices)
assert weights2.ndim == 1 and len(weights2) == len(mesh2.vertices)
weights = np.concatenate((weights1, weights2))

colours1 = mesh1.vertex_colors
colours2 = mesh2.vertex_colors
colours = np.concatenate([colours1, colours2], axis=0)

kd_tree = o3d.geometry.KDTreeFlann(points.T)

In [9]:
# ---------------------------------------------------------------------
# Normals
# ---------------------------------------------------------------------
mesh1.compute_vertex_normals()
mesh2.compute_vertex_normals()

normals1 = np.asarray(mesh1.vertex_normals)
normals2 = np.asarray(mesh2.vertex_normals)
normals = np.concatenate([normals1, normals2], axis=0)

scan_ids = np.concatenate([np.full(len(pts), i) for i, pts in enumerate(pointclouds)])

normals = smooth_normals(points, normals, tree=kd_tree, k=8, T=0.7, n_iters=nrm_smth_iters)

In [10]:
# ---------------------------------------------------------------------
# Local geometric properties
# ---------------------------------------------------------------------
local_spacing_1, local_density_1 = calc_local_spacing(points1, points1, tree=kd_tree)
local_spacing_2, local_density_2 = calc_local_spacing(points2, points2, tree=kd_tree)
local_spacings = (local_spacing_1, local_spacing_2)

local_spacing = np.concatenate(local_spacings)
local_density = np.concatenate((local_density_1, local_density_2))

global_avg_spacing = (1 / len(local_spacings)) * np.sum(
    [(1 / len(ls)) * np.sum(ls) for ls in local_spacings]
)

In [11]:
# ---------------------------------------------------------------------
# Overlap detection
# ---------------------------------------------------------------------
nbr_cache = precompute_cyl_neighbours(points, normals, local_spacing, r_alpha, h_alpha, kd_tree)

overlap_idx, overlap_mask = compute_overlap_set_cached(scan_ids, nbr_cache)
overlap_idx, overlap_mask = smooth_overlap_set_cached(overlap_mask, nbr_cache)
overlap_idx, overlap_mask = smooth_overlap_set_cached(overlap_mask, nbr_cache, p_thresh=1)

In [ ]:
debug_colours = np.array([
    [255,   0,   0],
    [  0, 255,   0],
])
f0 = overlap_mask
f1 = ~overlap_mask
filters = np.column_stack((f0, f1))
idx = filters.argmax(axis=1)
row_colours = debug_colours[idx]
# row_colours[oidx] = np.array([0,0,0])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(row_colours)

# o3d.visualization.draw_geometries([pcd])

In [45]:
m1 = scan_ids==0
m2 = overlap_mask
debug_colours = np.array([
    [255,   0,   0],
    [  0, 255,   0],
    [  0,   0, 255],
    [255,   0, 255],
])
f0 = ~m1 & ~m2
f1 =  m1 & ~m2
f2 = ~m1 &  m2
f3 =  m1 &  m2
filters = np.column_stack((f0, f1, f2, f3))
idx = filters.argmax(axis=1)
row_colours = debug_colours[idx]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(row_colours)

o3d.visualization.draw_geometries([pcd])

In [12]:
# ---------------------------------------------------------------------
# Find overlap boundary edges
# ---------------------------------------------------------------------
tris1 = np.asarray(mesh1.triangles)
tris2 = np.asarray(mesh2.triangles) + len(points1)  # shift indices
all_tris = np.concatenate([tris1, tris2], axis=0)

nonoverlap_tris = all_tris[~np.all(overlap_mask[all_tris], axis=1)]
boundary_edges = find_boundary_edges(nonoverlap_tris)

In [13]:
# ---------------------------------------------------------------------
# Update weights
# ---------------------------------------------------------------------

updated_weights = update_weights(points, normals, weights, overlap_mask, scan_ids, nbr_cache)

In [14]:
# ---------------------------------------------------------------------
# Multilateral point shifting along normals
# ---------------------------------------------------------------------
normal_shifted_points = points.copy()
for _ in range(nrm_shift_iters):
    normal_shifted_points = normal_shift_smooth(
        points=normal_shifted_points,
        normals=normals,
        weights=weights,
        local_spacing=local_spacing,
        local_density=local_density,
        overlap_idx=overlap_idx,
        nbr_cache=nbr_cache,
        normal_diff_thresh=45,
        r_alpha=r_alpha,
        h_alpha=h_alpha,
        sigma_theta=sigma_theta,
        shift_all=shift_all)

kd_tree = o3d.geometry.KDTreeFlann(normal_shifted_points.T)

print(np.mean(np.abs(points - normal_shifted_points)))

[WARN; vertex smoother] Using w_density=1.0 as sigma_rho < rho_floor.
[WARN; vertex smoother] Using w_density=1.0 as sigma_rho < rho_floor.
2.6989912717235817e-05


In [149]:
shift_comparison_colours = np.concatenate([np.full((len(normal_shifted_points), 3), [0, 1, 0], dtype=np.int32), np.full((len(points), 3), [1, 0, 0], dtype=np.int32)], axis=0)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.concatenate([normal_shifted_points, points], axis=0))
pcd.colors = o3d.utility.Vector3dVector(shift_comparison_colours)
o3d.visualization.draw_geometries([pcd])

In [155]:
shift = np.linalg.norm(points - normal_shifted_points, axis=1)
noshift_mask = shift==0
shift_rankings = np.argsort(np.argsort(shift[~noshift_mask]))
rankings = np.zeros_like(shift)
rankings[~noshift_mask] = shift_rankings
nrm_rankings = rankings/rankings.max()

shift_colours = (1-nrm_rankings[:,None])*np.full_like(points, np.array([0, 1, 1]))+nrm_rankings[:,None]*np.full_like(points, np.array([1, 0, 1]))
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(normal_shifted_points)
pcd.colors = o3d.utility.Vector3dVector(shift_colours)
o3d.visualization.draw_geometries([pcd])

In [15]:
# ---------------------------------------------------------------------
# Merge nearby clusters
# ---------------------------------------------------------------------
merged_out = merge_nearby_clusters(
    normal_shifted_points=normal_shifted_points,
    normals=normals,
    weights=updated_weights,
    colours=colours,
    overlap_mask=overlap_mask,
    overlap_idx=overlap_idx,
    global_avg_spacing=global_avg_spacing,
    h_alpha=h_alpha,
    tree=kd_tree,
)
cluster_mapping, clustered_overlap_pnts, clustered_overlap_cols, clustered_overlap_nrms, clustered_overlap_wts = merged_out

In [166]:
demo_pts = np.concatenate((clustered_overlap_pnts, normal_shifted_points))
demo_col = np.concatenate((np.full((len(clustered_overlap_pnts),3), [1, 0, 1]), np.full((len(normal_shifted_points),3), [0, 1, 1])))

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(demo_pts)
pcd.colors = o3d.utility.Vector3dVector(demo_col)
o3d.visualization.draw_geometries([pcd])

In [16]:
# ---------------------------------------------------------------------
# Classify vertices
# ---------------------------------------------------------------------
tri_has_overlap_any = overlap_mask[all_tris].any(axis=1)
overlap_any_idx = np.unique(all_tris[tri_has_overlap_any])

border_mask = np.zeros(len(points), dtype=bool)
border_mask[overlap_any_idx] = True
border_mask[cluster_mapping != -1] = False

nonoverlap_nonborder_mask = np.zeros(len(points), dtype=bool)
nonoverlap_nonborder_mask[cluster_mapping == -1] = True
nonoverlap_nonborder_mask[border_mask] = False

n_overlap = len(clustered_overlap_pnts)
n_border = border_mask.sum()
n_free = nonoverlap_nonborder_mask.sum()

new_points = np.concatenate(
    [
        clustered_overlap_pnts,
        normal_shifted_points[border_mask],
        normal_shifted_points[nonoverlap_nonborder_mask],
    ],
    axis=0,
)
new_colours = np.concatenate(
    [
        clustered_overlap_cols,
        colours[border_mask],
        colours[nonoverlap_nonborder_mask],
    ],
    axis=0,
)
new_normals = np.concatenate(
    [
        clustered_overlap_nrms,
        normals[border_mask],
        normals[nonoverlap_nonborder_mask],
    ],
    axis=0,
)
new_weights = np.concatenate(
        [
            clustered_overlap_wts,
            weights[border_mask],
            weights[nonoverlap_nonborder_mask],
        ],
        axis=0,
    )

new_colours = np.clip(new_colours, 0, 1)

In [17]:
# ---------------------------------------------------------------------
# Complete mapping
# ---------------------------------------------------------------------
border_idx_from = np.arange(len(points))[border_mask]
border_idx_to = np.arange(n_border) + n_overlap

free_idx_from = np.arange(len(points))[nonoverlap_nonborder_mask]
free_idx_to = np.arange(n_free) + n_overlap + n_border

mapping = cluster_mapping.copy()
mapping[border_idx_from] = border_idx_to
mapping[free_idx_from] = free_idx_to

In [18]:
def density_aware_radii(pcd: o3d.geometry.PointCloud, k=10):
    pts = np.asarray(pcd.points)
    kdt = o3d.geometry.KDTreeFlann(pcd)
    dk = np.empty(len(pts))
    for i, p in enumerate(pts):
        _, _, d2 = kdt.search_knn_vector_3d(p, k+1)  # includes the point itself
        dk[i] = np.sqrt(d2[-1])                      # k-th neighbor distance
    h10, h50, h90 = np.percentile(dk, [10, 50, 90])
    r_min = 0.8 * h10
    r_mid = h50
    r_max = 1.2 * h90
    return [r_min, r_mid, r_max]

In [19]:
# ---------------------------------------------------------------------
# Mesh the overlap zone
# ---------------------------------------------------------------------
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_points[:n_overlap])

if new_normals[:n_overlap] is not None:
    pcd.normals = o3d.utility.Vector3dVector(new_normals[:n_overlap])
else:
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=2.5 * global_avg_spacing, max_nn=30
        )
    )
    pcd.orient_normals_consistent_tangent_plane(k=30)

# r_min = global_avg_spacing
# radii = o3d.utility.DoubleVector(np.geomspace(r_min, r_min*4, num=5))

# points_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
# dists = points_pcd.compute_nearest_neighbor_distance()
# s = np.median(dists)

# radii = o3d.utility.DoubleVector([1.2*s, 2*s, 3*s, 4*s])

radii = o3d.utility.DoubleVector(density_aware_radii(pcd, k=10))
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(40))
pcd.orient_normals_consistent_tangent_plane(50)

overlap_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, radii
)

# overlap_mesh.remove_duplicated_vertices()
overlap_mesh.remove_duplicated_triangles()
overlap_mesh.remove_degenerate_triangles()
overlap_mesh.remove_non_manifold_edges()
overlap_mesh.compute_vertex_normals()

TriangleMesh with 2570 points and 4893 triangles.

In [171]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_points)
colours = np.concatenate([
    np.full((n_overlap,3),[255, 0, 0]),
    np.full((n_border,3), [0, 255, 0]),
    np.full((n_free,3),   [0, 0, 255]),
])
pcd.colors = o3d.utility.Vector3dVector(colours)

o3d.visualization.draw_geometries([overlap_mesh, pcd])

In [20]:
new_nonoverlap_tris = mapping[nonoverlap_tris]
nonoverlap_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(new_points),
    triangles=o3d.utility.Vector3iVector(new_nonoverlap_tris),
)

# o3d.visualization.draw_geometries([nonoverlap_mesh, pcd])

In [24]:
# o3d.visualization.draw_geometries([mesh1, mesh2, pcd])

In [21]:
# ---------------------------------------------------------------------
# Trim overlap mesh
# ---------------------------------------------------------------------
mapped_boundary_edges = mapping[boundary_edges]
relevant_boundary_edges = mapped_boundary_edges[
    np.all(mapped_boundary_edges < len(overlap_mesh.vertices), axis=1)
]

Ncc = len(overlap_mesh.cluster_connected_triangles()[1])

trimmed_overlap_mesh = topological_trim(
    overlap_mesh, relevant_boundary_edges, k=Ncc,
)

o3d.visualization.draw_geometries([trimmed_overlap_mesh, pcd])

In [22]:
def compact_by_faces(V, F, attrs):
    """
    V: (N,3), F: (M,3), attrs: list of (N,...) arrays (e.g. colors, normals, weights)
    Returns Vc, Fc, attrs_c (all compacted), used_mask, remap
    """
    used = np.zeros(len(V), dtype=bool)
    used[np.unique(F)] = True
    remap = -np.ones(len(V), dtype=int)
    remap[used] = np.arange(used.sum())
    Vc = V[used]
    Fc = remap[F]
    attrs_c = [a[used] if a is not None else None for a in attrs]
    return Vc, Fc, attrs_c, used, remap

In [ ]:
trimmed_overlap_tris = np.asarray(trimmed_overlap_mesh.triangles)
fused_mesh_triangles = np.concatenate(
    [trimmed_overlap_tris, mapping[nonoverlap_tris]], axis=0
)

V0, F0, (C0, N0, W0), used_mask, remap = compact_by_faces(
    new_points, fused_mesh_triangles, [new_colours, new_normals, new_weights]
)

fused_mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(V0),
    triangles=o3d.utility.Vector3iVector(F0),
)

fused_mesh.vertex_colors = o3d.utility.Vector3dVector(np.clip(C0, 0, 1))

fused_mesh.remove_duplicated_triangles()
fused_mesh.remove_degenerate_triangles()
fused_mesh.remove_non_manifold_edges()
# fused_mesh.remove_unreferenced_vertices()
# fused_mesh.remove_duplicated_vertices()
# fused_mesh.compute_vertex_normals()

TriangleMesh with 8470 points and 16593 triangles.

In [25]:
from superprimitive_fusion.mesh_fusion_utils import (
    colour_transfer,
)

fill_holes=False
if not fill_holes:
    fused_mesh.compute_vertex_normals()
else:
    V0, F0 = sanitise_mesh(new_points, fused_mesh_triangles)
    C0 = new_colours

    mf = pymeshfix.PyTMesh(False)
    mf.load_array(V0, F0)

    mf.fill_small_boundaries(nbe=20)

    V1, F1 = mf.return_arrays()

    C1 = colour_transfer(V0, C0, V1)

    repaired = o3d.geometry.TriangleMesh()
    repaired.vertices = o3d.utility.Vector3dVector(V1)
    repaired.triangles = o3d.utility.Vector3iVector(F1)
    repaired.vertex_colors = o3d.utility.Vector3dVector(C1)
    repaired.compute_vertex_normals()

In [176]:
o3d.visualization.draw_geometries([fused_mesh, pcd])

In [ ]:
# V0 = np.ascontiguousarray(new_points, dtype=np.float64)
# F0 = np.ascontiguousarray(fused_mesh.triangles, dtype=np.int32)
# C0 = new_colours

# # sanity checks
# if not np.isfinite(V0).all():
#     raise ValueError("Non-finite vertex coordinates")
# if F0.min() < 0 or F0.max() >= len(V0):
#     raise ValueError("Face indices out of range")

# V0, F0 = sanitise_mesh(new_points, fused_mesh_triangles)

# mf = pymeshfix.PyTMesh(False)
# mf.load_array(V0, F0)

# mf.fill_small_boundaries(nbe=10)

# V1, F1 = mf.return_arrays()

# # ---- colour transfer
# tree = cKDTree(V0)
# dist, idx = tree.query(V1, k=1)
# C1 = C0[idx]

# mask = dist > 1e-6
# if mask.any():
#     dist3, idx3 = tree.query(V1[mask], k=3)
#     w = 1.0 / (dist3 + 1e-12)
#     w /= w.sum(axis=1, keepdims=True)
#     C1[mask] = (C0[idx3] * w[..., None]).sum(axis=1)

# repaired = o3d.geometry.TriangleMesh()
# repaired.vertices = o3d.utility.Vector3dVector(V1)
# repaired.triangles = o3d.utility.Vector3iVector(F1)
# repaired.vertex_colors = o3d.utility.Vector3dVector(C1)
# repaired.compute_vertex_normals()

# o3d.visualization.draw_geometries([repaired])